In [78]:
from requests import get
import pandas as pd
import feedparser as fp
from haversine import haversine

In [4]:
LAT = 41.77
LON = -71.39

In [5]:
#download the buoy list (one time only)
#root = ET.fromstring(get('https://www.ndbc.noaa.gov/activestations.xml').text)
#buoy_list = pd.DataFrame(columns = root[0].attrib.keys())
#for entry in root:
#    buoy_list = buoy_list.append(entry.attrib, ignore_index = True)
#buoy_list.to_csv('NDBC_Buoy_List.csv', index = False)

In [6]:
#read in csv data
buoy_list = pd.read_csv('NDBC_Buoy_List.csv')
buoy_list = buoy_list[buoy_list["met"] == 'y']

In [7]:
#Create a function to find the closet buoy on NOAA's list
def find_closest_buoy(lat, lon, buoy_list):
    smallest = 20036
    for ind in buoy_list.index:
        buoy_lon = buoy_list['lon'][ind]
        buoy_lat = buoy_list['lat'][ind]
        distance = haversine(LON, LAT, buoy_lon, buoy_lat)
        if distance < smallest:
            smallest = distance
            closest = buoy_list['id'][ind]
    return closest

In [47]:
buoy_name = find_closest_buoy(LAT, LON, buoy_list)
buoy_name

'pvdr1'

In [68]:
buoy_data = fp.parse('https://www.ndbc.noaa.gov/data/latest_obs/' + buoy_name + '.rss')

In [69]:
entry = buoy_data.entries[0]
print(entry.keys())

dict_keys(['published', 'published_parsed', 'title', 'title_detail', 'summary', 'summary_detail', 'links', 'link', 'id', 'guidislink', 'where'])


In [70]:
#root = ET.fromstring(get('https://www.ndbc.noaa.gov/activestations.xml').text)
#buoy_list = pd.DataFrame(columns = root[0].attrib.keys())
#for entry in root:
#    buoy_list = buoy_list.append(entry.attrib, ignore_index = True)

In [71]:
entry.summary

'<strong>December 14, 2020 8:00 am EST</strong><br />\n        <strong>Location:</strong> 41.786N 71.383W<br />\n        <strong>Wind Direction:</strong> NNW (340&#176;)<br />\n        <strong>Wind Speed:</strong> 8.9 knots<br />\n        <strong>Wind Gust:</strong> 13.0 knots<br />\n        <strong>Atmospheric Pressure:</strong> 30.00 in (1015.9 mb)<br />\n        <strong>Pressure Tendency:</strong> +0.04 in (+1.3 mb)<br />\n        <strong>Visibility:</strong> 5.9 nmi<br />'

In [64]:
#buoy_data = get('https://www.ndbc.noaa.gov/data/latest_obs/' + buoy_name + '.rss')

In [67]:
#buoy_data.content

b'<?xml version="1.0"?>\n<?xml-stylesheet type="text/xsl" href="/rss/ndbcrss.xsl"?>\n<rss version="2.0" xmlns:georss="http://www.georss.org/georss" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:atom="http://www.w3.org/2005/Atom">\n  <channel>\n    <title>NDBC - Station PVDR1 - 8453662 - Providence Visibility, RI Observations</title>\n    <description><![CDATA[This feed shows recent marine weather observations from Station PVDR1.]]></description>\n    <link>https://www.ndbc.noaa.gov/</link>\n    <pubDate>Mon, 14 Dec 2020 13:35:25 +0000</pubDate>\n    <lastBuildDate>Mon, 14 Dec 2020 13:35:25 +0000</lastBuildDate>\n    <ttl>30</ttl>\n    <language>en-us</language>\n    <managingEditor>webmaster.ndbc@noaa.gov (NDBC Webmaster)</managingEditor>\n    <webMaster>webmaster.ndbc@noaa.gov (NDBC Webmaster)</webMaster>\n    <image>\n      <url>https://www.ndbc.noaa.gov/images/noaa_nws_xml_logo.gif</url>\n      <title>NDBC - Station PVDR1 - 8453662 - Providence Visibility, RI Observations</title

In [72]:
buoy_raw = entry.summary.replace('<strong>', '').replace('</strong>', '')
buoy_raw

'December 14, 2020 8:00 am EST<br />\n        Location: 41.786N 71.383W<br />\n        Wind Direction: NNW (340&#176;)<br />\n        Wind Speed: 8.9 knots<br />\n        Wind Gust: 13.0 knots<br />\n        Atmospheric Pressure: 30.00 in (1015.9 mb)<br />\n        Pressure Tendency: +0.04 in (+1.3 mb)<br />\n        Visibility: 5.9 nmi<br />'

In [73]:
#Timestamp
ts_end = buoy_raw.index('<br />')
buoy_ts = buoy_raw[0:ts_end]
buoy_ts

'December 14, 2020 8:00 am EST'

In [75]:
#location
loc_start = buoy_raw.index('Location: ') + 10
loc_end = buoy_raw[loc_start:].index('<br />') + loc_start
buoy_loc = buoy_raw[loc_start:loc_end]
buoy_loc

'41.786N 71.383W'

In [79]:
buoy_loc_lat = float(buoy_loc[0:buoy_loc.index('N')])
buoy_loc_lon = float(buoy_loc[buoy_loc.index(' ')+1:-1])*-1
buoy_dist = round(haversine(LON, LAT, buoy_loc_lon, buoy_loc_lat), 1)
buoy_dist

1.9

In [80]:
#Wind Direction
wd_start = buoy_raw.index('Wind Direction: ') + 16
wd_end = buoy_raw[wd_start:].index('<br />') + wd_start
buoy_wd = buoy_raw[wd_start:wd_end]
buoy_wd_deg = float(buoy_wd[buoy_wd.index('(')+1:buoy_wd.index('&')])
buoy_wd_deg

340.0

In [81]:
#Wind Speed
wspd_start = buoy_raw.index('Wind Speed: ') + 12
wspd_end = buoy_raw[wspd_start:].index('<br />') + wspd_start
buoy_wspd = buoy_raw[wspd_start:wspd_end]
buoy_wspd_kts = float(buoy_wspd[0:buoy_wspd.index(' ')])
buoy_wspd_kts

8.9

In [82]:
#Wind Gust
wgst_start = buoy_raw.index('Wind Gust: ') + 11
wgst_end = buoy_raw[wgst_start:].index('<br />') + wgst_start
buoy_wgst = buoy_raw[wgst_start:wgst_end]
buoy_wgst_kts = float(buoy_wgst[0:buoy_wgst.index(' ')])
buoy_wgst_kts

13.0

In [83]:
#Atmospheric Pressure
atm_start = buoy_raw.index('Atmospheric Pressure: ') + 22
atm_end = buoy_raw[atm_start:].index('<br />') + atm_start
buoy_atm = buoy_raw[atm_start:atm_end]
buoy_atm_mb = float(buoy_atm[buoy_atm.index('(')+1:buoy_atm.index(' mb')])
buoy_atm_mb

1015.9

In [84]:
#Air Temperature
temp_start = buoy_raw.index('Air Temperature: ') + 17
temp_end = buoy_raw[temp_start:].index('<br />') + temp_start
buoy_temp = buoy_raw[temp_start:temp_end]
buoy_temp_F = float(buoy_temp[0:buoy_temp.index('&')])
buoy_temp_F

ValueError: substring not found

In [87]:
try:
    buoy_raw.index('Air Temperature: ')
except ValueError:
    buoy_temp_F = None
print(buoy_temp_F)

None


In [41]:
#Dew Point
dpt_start = buoy_raw.index('Dew Point: ') + 11
dpt_end = buoy_raw[dpt_start:].index('<br />') + dpt_start
buoy_dpt = buoy_raw[dpt_start:dpt_end]
buoy_dpt_F = float(buoy_dpt[0:buoy_temp.index('&')])
buoy_dpt_F

38.8

In [42]:
#Visibility
viz_start = buoy_raw.index('Visibility: ') + 12
viz_end = buoy_raw[viz_start:].index('<br />') + viz_start
buoy_viz = buoy_raw[viz_start:viz_end]
buoy_viz_nm = float(buoy_viz[0:buoy_viz.index(' ')])
buoy_viz_nm

5.9

In [43]:
buoy_dict = {
    "Name": buoy_name +' (' + str(buoy_dist) + ' nm away)',
    "Timestamp": buoy_ts,
    "Location": buoy_loc,
    "Wind Direction (o)": buoy_wd_deg,
    "Wind Speeds (kts)": buoy_wspd_kts,
    "Wind Gusts (kts)": buoy_wgst_kts,
    "Pressure (mb)": buoy_atm_mb,
    "Temperature (F)": buoy_temp_F,
    "Dew Point (F)": buoy_dpt_F,
    "Visibility (nm)": buoy_viz_nm
}

In [44]:
buoy_dict

{'Name': 'pvdr1 (1.9 nm away)',
 'Timestamp': 'December 13, 2020 8:54 pm EST',
 'Location': '41.786N 71.383W',
 'Wind Direction (o)': 320.0,
 'Wind Speeds (kts)': 7.0,
 'Wind Gusts (kts)': 8.9,
 'Pressure (mb)': 1015.5,
 'Temperature (F)': 48.4,
 'Dew Point (F)': 38.8,
 'Visibility (nm)': 5.9}

# Turn into Function that can be  called

In [88]:
#To do:
#add error handeling for import table, with code to generate it if there is an issue

from requests import get
import pandas as pd
import feedparser as fp
from haversine import haversine

def noaa_buoy(LAT, LON):
    
    #read in csv data
    buoy_list = pd.read_csv('NDBC_Buoy_List.csv')
    buoy_list = buoy_list[buoy_list["met"] == 'y']
    
    #find the closet buoy on NOAA's list
    smallest = 21369              #circumference of earth of nm 
    for ind in buoy_list.index:
        buoy_lon = buoy_list['lon'][ind]
        buoy_lat = buoy_list['lat'][ind]
        distance = haversine(LON, LAT, buoy_lon, buoy_lat)
        if distance < smallest:
            smallest = distance
            closest = buoy_list['id'][ind]
    buoy_name = closest
    buoy_dist = round(float(smallest),1)
    
    #scrape the buoy data from NOAA's website
    buoy_data = fp.parse('https://www.ndbc.noaa.gov/data/latest_obs/' + buoy_name + '.rss')
    entry = buoy_data.entries[0]
    buoy_raw = entry.summary.replace('<strong>', '').replace('</strong>', '')
    
    #Timestamp
    ts_end = buoy_raw.index('<br />')
    buoy_ts = buoy_raw[0:ts_end]
    
    #location
    loc_start = buoy_raw.index('Location: ') + 10
    loc_end = buoy_raw[loc_start:].index('<br />') + loc_start
    buoy_loc = buoy_raw[loc_start:loc_end]
    
    #wind direction
    try:
        wd_start = buoy_raw.index('Wind Direction: ') + 16
        wd_end = buoy_raw[wd_start:].index('<br />') + wd_start
        buoy_wd = buoy_raw[wd_start:wd_end]
        buoy_wd_deg = float(buoy_wd[buoy_wd.index('(')+1:buoy_wd.index('&')])
    except:
        buoy_wd_deg = None
    
    #Wind Speed
    try:
        wspd_start = buoy_raw.index('Wind Speed: ') + 12
        wspd_end = buoy_raw[wspd_start:].index('<br />') + wspd_start
        buoy_wspd = buoy_raw[wspd_start:wspd_end]
        buoy_wspd_kts = float(buoy_wspd[0:buoy_wspd.index(' ')])
    except: buoy_wspd_kts = None
    
    #Wind Gust
    try:
        wgst_start = buoy_raw.index('Wind Gust: ') + 11
        wgst_end = buoy_raw[wgst_start:].index('<br />') + wgst_start
        buoy_wgst = buoy_raw[wgst_start:wgst_end]
        buoy_wgst_kts = float(buoy_wgst[0:buoy_wgst.index(' ')])
    except: buoy_wgst_kts = None

    #Atmospheric Pressure
    try:
        atm_start = buoy_raw.index('Atmospheric Pressure: ') + 22
        atm_end = buoy_raw[atm_start:].index('<br />') + atm_start
        buoy_atm = buoy_raw[atm_start:atm_end]
        buoy_atm_mb = float(buoy_atm[buoy_atm.index('(')+1:buoy_atm.index(' mb')])
    except: buoy_atm_mb = None

    #Air Temperature
    try:
        temp_start = buoy_raw.index('Air Temperature: ') + 17
        temp_end = buoy_raw[temp_start:].index('<br />') + temp_start
        buoy_temp = buoy_raw[temp_start:temp_end]
        buoy_temp_F = float(buoy_temp[0:buoy_temp.index('&')])
    except: buoy_temp_F = None

    #Dew Point
    try:
        dpt_start = buoy_raw.index('Dew Point: ') + 11
        dpt_end = buoy_raw[dpt_start:].index('<br />') + dpt_start
        buoy_dpt = buoy_raw[dpt_start:dpt_end]
        buoy_dpt_F = float(buoy_dpt[0:buoy_temp.index('&')])
    except: buoy_dpt_F = None
    
    #Visibility
    try:
        viz_start = buoy_raw.index('Visibility: ') + 12
        viz_end = buoy_raw[viz_start:].index('<br />') + viz_start
        buoy_viz = buoy_raw[viz_start:viz_end]
        buoy_viz_nm = float(buoy_viz[0:buoy_viz.index(' ')])
    except: buoy_viz_nm = None
    
    buoy_dict = {
    "Name": buoy_name +' (' + str(buoy_dist) + ' nm away)',
    "Timestamp": buoy_ts,
    "Location": buoy_loc,
    "Wind Direction (o)": buoy_wd_deg,
    "Wind Speeds (kts)": buoy_wspd_kts,
    "Wind Gusts (kts)": buoy_wgst_kts,
    "Pressure (mb)": buoy_atm_mb,
    "Temperature (F)": buoy_temp_F,
    "Dew Point (F)": buoy_dpt_F,
    "Visibility (nm)": buoy_viz_nm
    }
    
    return buoy_dict

In [90]:
CURRENT_LAT = 41.77
CURRENT_LON = -71.39
test_snapshot = noaa_buoy(CURRENT_LAT, CURRENT_LON)
print(test_snapshot) 

{'Name': '23009 (3472 nm away)', 'Timestamp': 'December 14, 2020 1300 UTC', 'Location': '15N 90E', 'Wind Direction (o)': 43.0, 'Wind Speeds (kts)': 14.8, 'Wind Gusts (kts)': None, 'Pressure (mb)': 1011.3, 'Temperature (F)': 81.7, 'Dew Point (F)': None, 'Visibility (nm)': None}
